#### Thresholds for Dropping vs Imputing Missing Data
* **Less than 5% Missing**
    * Drop the rows or columns

* **5% to 20% Missing**
    * Impute using averages, medians, or other simple methods.

* **More than 20% Missing**
    * Advanced Imputation techniques (KNN, regression, or ML-based models) should be considered.

* **Above 40% to 50% Missing**
    * Evaluate Necessity of Feature.
    * If the feature is crucial, try advanced imputation. Otherwise, consider dropping the entire column.

In [278]:
import pandas as pd
from sklearn.impute import KNNImputer

In [279]:
titanic_df = pd.read_csv('../data/Titanic-Dataset.csv')

In [284]:
class CleanseData:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
    
    def snapshot(self):
        """
        This method is used to provide a snapshot of the first few rows of data
        """
        return self.df.head()
    
    def describe_data(self):
        """
        This method is used to provide additional information and statistics about the dataset
        """
        return self.df.info(), self.df.describe()
    
    @staticmethod
    def define_categories(null_percentage: float):
        """
        This method is used to classify columns with missing values into various groups based on the percentage of missing values in the column
        """
        if null_percentage < .05:
            return "drop_row"
        elif null_percentage < .2:
            return "impute_median"
        elif null_percentage < .45:
            return "impute_knn"
        else:
            return "drop_column"
        
    def drop_row(self, column_name: str, index_list: list):
        """
        This method is used to drop rows from the df when missing column values are less than 5%
        """
        for index in index_list:
            self.df.drop(index, inplace=True)
            
        print(f" - Dropped {len(index_list)} rows from the '{column_name}' column of the dataframe\n")
            
        return self.df
    
    def impute_median(self, column_name: str, index_list: list):
        """
        This method is used to fill NaN values with the median of the column when missing column values are less than 20% and greater than 5%
        """
        median_value = self.df[column_name].median()
        self.df[column_name] = self.df[column_name].fillna(median_value)
        
        print(f" - Filled {len(index_list)} NaN values from column '{column_name}' with median value {median_value}\n") 
        
        return self.df
    
    def impute_knn(self, column_name: str, index_list: list):
        """
        This method imputes NaN values for specific rows in a column using K-Nearest Neighbors.
        """
        # Extract the relevant column for imputation
        temp_df = self.df[[column_name]].copy()
    
        # Apply KNN Imputer to the entire column
        imputer = KNNImputer(n_neighbors=3)
        imputed_values = imputer.fit_transform(temp_df)
    
        # Convert to DataFrame to preserve index
        imputed_df = pd.DataFrame(imputed_values, columns=[column_name], index=temp_df.index)
    
        # Update only the rows specified by index_list
        self.df.loc[index_list, column_name] = imputed_df.loc[index_list, column_name]
        
        print(f" - Imputed {len(index_list)} NaN values from column '{column_name}' using K-Nearest neighbors algorithm\n")
    
        return self.df
    
    def drop_column(self, column_name: str, index_list: list):
        """
        This method is used to drop columns from the df when missing column values are greater than 45%
        """
        self.df.drop(columns=[column_name], inplace=True)
        
        print(f" - Dropped column '{column_name}' containing {len(index_list)} NaN values\n")
        
        return self.df
    
    def handle_null(self):
        """
        This method is used to handle NaN values in the dataframe
        """
        # Drop duplicates
        print(f"DROPPING DUPLICATES...\n")
        before_drop = len(self.df)
        self.df = self.df.drop_duplicates()
        after_drop = len(self.df)
        print(f"* Dropped {before_drop - after_drop} duplicate rows from dataframe.\n\n")
        
        print(f"BEGINNING TO HANDLE NULL VALUES...\n")
        non_zero_dict = {}
        
        na_percentage = (self.df.isnull().sum() / len(self.df))
        non_zero_na = na_percentage[na_percentage > 0]
        
        for index, value in non_zero_na.items():
            non_zero_dict[index] = {
                "null_percent": value,
                "category": CleanseData.define_categories(value),
                "indexes": self.df[self.df[index].isnull()].index.tolist()
            }
            print(f"* Column '{index}' has {value * 100:.2f}% null values.\n")
        
        for key, value in non_zero_dict.items():
            if value['category'] == "drop_row":
                self.drop_row(column_name=key, index_list=value['indexes'])
            elif value['category'] == "impute_median":
                self.impute_median(column_name=key, index_list=value['indexes'])
            elif value['category'] == "impute_knn":
                self.impute_knn(column_name=key, index_list=value['indexes'])
            else:
                self.drop_column(column_name=key, index_list=value['indexes'])
        
    
    def get_report(self):
        self.handle_null()
        return self.df

In [290]:
cleanser = CleanseData(titanic_df)

cleanser.get_report()

DROPPING DUPLICATES...

* Dropped 0 duplicate rows from dataframe.


BEGINNING TO HANDLE NULL VALUES...

* Column 'Age' has 19.87% null values.

* Column 'Cabin' has 77.10% null values.

* Column 'Embarked' has 0.22% null values.

 - Filled 177 NaN values from column 'Age' with median value 28.0

 - Dropped column 'Cabin' containing 687 NaN values

 - Dropped 2 rows from the 'Embarked' column of the dataframe


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C
